In [0]:
import pyspark.sql.functions as sf
from pyspark.sql.types import *
from time import time
spark.conf.set("spark.sql.session.timeZone", "America/New_York")

In [0]:
# set time window for filtering data
# 1209600 = 2 weeks
unix_time_window = time() - 172800

In [0]:
df_skill_inputs_enriched = spark.read.table("runescape.02_silver.skill_action_inputs_enriched")

In [0]:

df_skill_xp_enriched = spark.read.table("runescape.02_silver.skill_action_xp_enriched")

In [0]:
# Read data from 'runescape.02_silver.item_mapping'
df_map = spark.read.table("runescape.02_silver.item_mapping")
df_map.display()

In [0]:
df_skill_xp_enriched.display()
df_skill_inputs_enriched.display()

In [0]:
# Read data from
df_1h_prices_enriched = spark.read.table("runescape.02_silver.1h_prices_enriched").filter(f"time >= {unix_time_window}")

In [0]:
df_1h_prices_enriched.display()

In [0]:
# combine input data with price data
df_1h_prices_enriched_w_inputs = df_1h_prices_enriched.join(df_skill_inputs_enriched,
                                            df_1h_prices_enriched.id == df_skill_inputs_enriched.id_output)
df_1h_prices_enriched_w_inputs = df_1h_prices_enriched_w_inputs.drop('id','name')
df_input_cost_test = df_1h_prices_enriched_w_inputs.filter("id_output = 22284").sort("time").display()

In [0]:
#

In [0]:
%skip
df_input_cost = df_1h_prices_enriched.join(df_skill_inputs_enriched,
                                            df_1h_prices_enriched.id == df_skill_inputs_enriched.id_input,
                                            "right")
df_input_cost = df_input_cost.withColumns({'input_cost_low': sf.col("avg1HourLow")*sf.col("input_qty"),
                                           'input_cost_high': sf.col("avg1HourHigh")*sf.col("input_qty")})

df_input_cost_test = df_input_cost.filter("id_output = 22284").sort("time").display()

The approch below doesn't work when the input has missing price data...
could use an outer join?

In [0]:
# The above can be used with an summation to get the input cost
df_input_total_cost = df_input_cost.groupBy("time","output", "skill").sum("input_cost_high", "input_cost_low")
df_input_total_cost = df_input_total_cost.withColumnsRenamed({"sum(input_cost_high)": "total_input_cost_high"
                                                              , "sum(input_cost_low)": "total_input_cost_low"})

df_input_total_cost.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df_profit = df_input_total_cost.join(df_hourly, [df_hourly.time == df_input_total_cost.time, df_hourly.name == df_input_total_cost.output])
# .98 tax
df_profit = df_profit.withColumn("low_profit", sf.col("avg1HourHigh")*.98 - sf.col("total_input_cost_high"))
# .98 tax
df_profit = df_profit.withColumn("high_profit", sf.col("avg1HourLow")*.98 - sf.col("total_input_cost_low"))
df_profit.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df_profit_ruby_unstrung = df_profit.filter("output == 'Ruby amulet (u)'")
df_profit_ruby_unstrung.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df_profit_all = df_profit
df_profit_all.display()


Databricks visualization. Run in Databricks to view.